In [0]:
from pyspark.sql import functions as F

T_GOLD_GENRE_SCORE = f"{CATALOG}.{SCHEMA}.gold_genre_score"

MIN_VOTES = 500

movies = spark.table(T_SILVER_MOVIES)
genres = spark.table(T_SILVER_GENRES)
bridge = spark.table(T_SILVER_MOVIE_GENRES)

df_gold_genre_score = (
    bridge
    .join(movies, "movie_id", "inner")
    .join(genres, "genre_id", "inner")
    .filter(F.col("vote_count") >= MIN_VOTES)
    .groupBy("genre_id", "genre_name")
    .agg(
        F.countDistinct("movie_id").alias("movies_considered"),
        F.sum("vote_count").alias("total_votes"),
        (F.sum(F.col("vote_average") * F.col("vote_count")) / F.sum("vote_count")).alias("weighted_rating")
    )
    .orderBy(F.col("weighted_rating").desc())
)

display(df_gold_genre_score)

df_gold_genre_score.write.format("delta").mode("overwrite").saveAsTable(T_GOLD_GENRE_SCORE)


genre_id,genre_name,movies_considered,total_votes,weighted_rating
80,Crime,92,1023066,7.751937052936954
18,Drama,288,2499797,7.709062326660922
10752,War,63,439391,7.607020328135988
36,History,71,427781,7.602154925066798
16,Animation,84,866181,7.591992341092682
53,Thriller,126,1434843,7.523572294669172
878,Science Fiction,111,1709907,7.466642977659017
9648,Mystery,67,639889,7.439574233968707
12,Adventure,208,2744371,7.413032949626706
35,Comedy,188,1542624,7.412460338358536


In [0]:
best_row = df_gold_genre_score.select("genre_id", "genre_name").first()
best_genre_id = best_row["genre_id"]
best_genre_name = best_row["genre_name"]
print(f"✅ Best genre to invest: {best_genre_name} (id={best_genre_id})")


✅ Best genre to invest: Crime (id=80)


In [0]:
from pyspark.sql.window import Window

T_GOLD_TOP_MOVIES_BEST_GENRE = f"{CATALOG}.{SCHEMA}.gold_top_movies_best_genre"
TOP_N = 10

w = Window.orderBy(
    F.col("vote_average").desc(),
    F.col("vote_count").desc(),
    F.col("popularity").desc()
)

df_gold_top_movies_best_genre = (
    bridge
    .join(movies, "movie_id", "inner")
    .filter((F.col("genre_id") == best_genre_id) & (F.col("vote_count") >= MIN_VOTES))
    .withColumn("rank", F.row_number().over(w))
    .filter(F.col("rank") <= TOP_N)
    .select(
        "movie_id", "title", "release_date",
        "vote_average", "vote_count", "popularity",
        "original_language", "rank"
    )
    .orderBy("rank")
)

display(df_gold_top_movies_best_genre)

df_gold_top_movies_best_genre.write.format("delta").mode("overwrite").saveAsTable(T_GOLD_TOP_MOVIES_BEST_GENRE)


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


movie_id,title,release_date,vote_average,vote_count,popularity,original_language,rank
278,The Shawshank Redemption,1994-09-23,8.715,29570,32.2333,en,1
238,The Godfather,1972-03-14,8.686,22330,33.8438,en,2
240,The Godfather Part II,1974-12-20,8.57,13503,18.6727,en,3
155,The Dark Knight,2008-07-16,8.525,35024,23.4095,en,4
497,The Green Mile,1999-12-10,8.503,18744,20.9725,en,5
680,Pulp Fiction,1994-09-10,8.486,29563,19.5594,en,6
769,GoodFellas,1990-09-12,8.452,13934,16.5418,en,7
807,Se7en,1995-09-22,8.379,22498,18.859,en,8
274,The Silence of the Lambs,1991-02-14,8.346,17477,3.1778,en,9
101,Léon: The Professional,1994-09-14,8.298,15712,10.9992,fr,10


In [0]:
print("gold_genre_score:", spark.table(T_GOLD_GENRE_SCORE).count())
print("gold_top_movies_best_genre:", spark.table(T_GOLD_TOP_MOVIES_BEST_GENRE).count())


gold_genre_score: 19
gold_top_movies_best_genre: 10


In [0]:
T_GOLD_TOP_MOVIES_BY_GENRE = f"{CATALOG}.{SCHEMA}.gold_top_movies_by_genre"
TOP_N = 10

df_all = (
    bridge
    .join(movies, "movie_id", "inner")
    .join(genres, "genre_id", "inner")
    .filter(F.col("vote_count") >= MIN_VOTES)
)

w = Window.partitionBy("genre_id").orderBy(
    F.col("vote_average").desc(),
    F.col("vote_count").desc(),
    F.col("popularity").desc()
)

df_gold_top_by_genre = (
    df_all
    .withColumn("rank_in_genre", F.row_number().over(w))
    .filter(F.col("rank_in_genre") <= TOP_N)
    .select(
        "genre_id","genre_name",
        "movie_id","title","release_date",
        "vote_average","vote_count","popularity","original_language",
        "rank_in_genre"
    )
)

display(df_gold_top_by_genre)

df_gold_top_by_genre.write.format("delta").mode("overwrite").saveAsTable(T_GOLD_TOP_MOVIES_BY_GENRE)


genre_id,genre_name,movie_id,title,release_date,vote_average,vote_count,popularity,original_language,rank_in_genre
12,Adventure,122,The Lord of the Rings: The Return of the King,2003-12-17,8.492,25894,23.125,en,1
12,Adventure,157336,Interstellar,2014-11-05,8.466,38670,45.7256,en,2
12,Adventure,121,The Lord of the Rings: The Two Towers,2002-12-18,8.412,23308,18.0537,en,3
12,Adventure,120,The Lord of the Rings: The Fellowship of the Ring,2001-12-18,8.4,26859,26.2427,en,4
12,Adventure,324857,Spider-Man: Into the Spider-Verse,2018-12-06,8.4,16769,15.5413,en,5
12,Adventure,1891,The Empire Strikes Back,1980-05-20,8.392,17992,9.1634,en,6
12,Adventure,4935,Howl's Moving Castle,2004-09-09,8.388,10758,15.4232,ja,7
12,Adventure,27205,Inception,2010-07-15,8.37,38533,26.1246,en,8
12,Adventure,105,Back to the Future,1985-07-03,8.323,21294,17.3559,en,9
12,Adventure,128,Princess Mononoke,1997-07-12,8.3,8712,10.3336,ja,10


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

CATALOG = "tmdb"
SCHEMA = "default"

# Silver tables (Unity Catalog)
T_SILVER_MOVIES = f"{CATALOG}.{SCHEMA}.silver_movies"
T_SILVER_GENRES = f"{CATALOG}.{SCHEMA}.silver_genres"
T_SILVER_MOVIE_GENRES = f"{CATALOG}.{SCHEMA}.silver_movie_genres"

# Gold tables
T_GOLD_GENRE_SCORE = f"{CATALOG}.{SCHEMA}.gold_genre_score"
T_GOLD_TOP_MOVIES_BEST_GENRE = f"{CATALOG}.{SCHEMA}.gold_top_movies_best_genre"

TOP_N = 10
MIN_VOTES = 500

# 1) Best genre (por weighted_rating)
best_row = (
    spark.table(T_GOLD_GENRE_SCORE)
    .orderBy(F.col("weighted_rating").desc())
    .select("genre_id", "genre_name")
    .first()
)

if best_row is None:
    raise Exception("❌ gold_genre_score está vacío. Revisa que la tabla exista y tenga datos.")

best_id = best_row["genre_id"]
best_name = best_row["genre_name"]
print(f"✅ Best genre: {best_name} (id={best_id})")

# 2) Load Silver
movies = spark.table(T_SILVER_MOVIES)
genres = spark.table(T_SILVER_GENRES)
bridge = spark.table(T_SILVER_MOVIE_GENRES)

# 3) Window con partitionBy para evitar warning
w = Window.partitionBy("genre_id").orderBy(
    F.col("vote_average").desc(),
    F.col("vote_count").desc(),
    F.col("popularity").desc()
)

# 4) Build Gold (Top N movies del mejor género) incluyendo genre_id y genre_name
df = (
    bridge.join(movies, "movie_id", "inner")
          .join(genres, "genre_id", "inner")
          .filter((F.col("genre_id") == best_id) & (F.col("vote_count") >= MIN_VOTES))
          .withColumn("rank", F.row_number().over(w))
          .filter(F.col("rank") <= TOP_N)
          .select(
              "genre_id", "genre_name",
              "movie_id", "title", "release_date",
              "vote_average", "vote_count", "popularity",
              "original_language", "rank"
          )
          .orderBy("rank")
)

display(df)

# 5) Save as Delta table in Unity Catalog (permitiendo evolución de schema)
(df.write.format("delta")
   .mode("overwrite")
   .option("overwriteSchema", "true")   # fuerza reemplazo de schema si aplica
   .option("mergeSchema", "true")       # permite agregar columnas nuevas
   .saveAsTable(T_GOLD_TOP_MOVIES_BEST_GENRE)
)

print("✅ Saved:", T_GOLD_TOP_MOVIES_BEST_GENRE)


✅ Best genre: Crime (id=80)


genre_id,genre_name,movie_id,title,release_date,vote_average,vote_count,popularity,original_language,rank
80,Crime,278,The Shawshank Redemption,1994-09-23,8.715,29570,32.2333,en,1
80,Crime,238,The Godfather,1972-03-14,8.686,22330,33.8438,en,2
80,Crime,240,The Godfather Part II,1974-12-20,8.57,13503,18.6727,en,3
80,Crime,155,The Dark Knight,2008-07-16,8.525,35024,23.4095,en,4
80,Crime,497,The Green Mile,1999-12-10,8.503,18744,20.9725,en,5
80,Crime,680,Pulp Fiction,1994-09-10,8.486,29563,19.5594,en,6
80,Crime,769,GoodFellas,1990-09-12,8.452,13934,16.5418,en,7
80,Crime,807,Se7en,1995-09-22,8.379,22498,18.859,en,8
80,Crime,274,The Silence of the Lambs,1991-02-14,8.346,17477,3.1778,en,9
80,Crime,101,Léon: The Professional,1994-09-14,8.298,15712,10.9992,fr,10


✅ Saved: tmdb.default.gold_top_movies_best_genre
